<a href="https://colab.research.google.com/github/bhaveshprajapat/dissertation-colab-gpt2/blob/main/Language%20Modelling%20in%20Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Language Modelling in Colab Pro
Based off of code from Max Woolf (https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce)

In [ ]:
%tensorflow_version 1.x

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

TensorFlow 1.x selected.
Sun May 10 16:10:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                   

# Setup
Install the PIP requirements and mount the Google Drive

In [ ]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Runtime Constants
model_size is one of:
*   124M (S)
*   355M (M)
*   774M (L)
*   355M (XL)



In [ ]:
model_size = "124M"
sess_run_name = "GPT2S-A-SATIRE-500" 

In [ ]:
gpt2.copy_checkpoint_from_gdrive("GPT2L-A-SATIRE-500" )

# Download Satire Datasets
Downloads the ready made satire dataset from the *GitLab*

In [ ]:
# Comment and uncomment as necessary
!rm -rf DatasetA
!cp "/content/drive/My Drive/DatasetA.zip" .
!unzip -q DatasetA.zip -d DatasetA

In [ ]:
#!tar -xf /content/satire-datasets/satire-train.tar

In [ ]:
# Set the dataset file name
folder_name = "/content/DatasetA"

# Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
gpt2.download_gpt2(model_name=model_size)
import tensorflow as tf
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
#gpt2.load_gpt2(sess, run_name='GPT2L-A-SATIRE-500')
gpt2.finetune(sess,
              dataset=folder_name,
              steps=500,
              restore_from='fresh',
              run_name="GPT2S-A-SATIRE-500",
              print_every=10,
              model_name="124M",
              sample_every=500,
              save_every=1000
              )
gpt2.copy_checkpoint_to_gdrive(run_name=sess_run_name)

Fetching checkpoint: 1.05Mit [00:00, 297Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 93.4Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 848Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 234Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 292Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 144Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 156Mit/s]                                                       


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/26810 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 26810/26810 [00:59<00:00, 448.83it/s]


dataset has 10178271 tokens
Training...
[10 | 21.11] loss=3.09 avg=3.09
[20 | 33.75] loss=3.20 avg=3.15
[30 | 46.49] loss=3.34 avg=3.21
[40 | 59.19] loss=3.16 avg=3.20
[50 | 71.86] loss=3.21 avg=3.20
[60 | 84.55] loss=3.13 avg=3.19
[70 | 97.20] loss=3.28 avg=3.20
[80 | 109.94] loss=3.12 avg=3.19
[90 | 122.62] loss=3.25 avg=3.20
[100 | 135.29] loss=3.24 avg=3.20
[110 | 148.02] loss=3.10 avg=3.19
[120 | 160.67] loss=2.89 avg=3.17
[130 | 173.32] loss=3.31 avg=3.18
[140 | 185.94] loss=3.21 avg=3.18
[150 | 198.58] loss=3.12 avg=3.18
[160 | 211.22] loss=3.26 avg=3.18
[170 | 223.89] loss=3.38 avg=3.20
[180 | 236.55] loss=3.20 avg=3.20
[190 | 249.15] loss=3.01 avg=3.19
[200 | 261.75] loss=3.13 avg=3.18
[210 | 274.42] loss=3.36 avg=3.19
[220 | 287.08] loss=3.34 avg=3.20
[230 | 299.75] loss=3.55 avg=3.22
[240 | 312.38] loss=3.23 avg=3.22
[250 | 324.98] loss=3.04 avg=3.21
[260 | 337.64] loss=3.08 avg=3.20
[270 | 350.33] loss=3.16 avg=3.20
[280 | 362.97] loss=3.09 avg=3.20
[290 | 375.66] loss=2.91

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name="GPT2L-A-SATIRE-5K")

KeyboardInterrupt: ignored

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name="GPT2L-A-SATIRE-3K")

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name="GPT2L-A-SATIRE-2K")

# Generate samples

In [ ]:
# Copy a checkpoint to the Colab Runtime
#!cp -r "drive/My Drive/Saved Colab Checkpoints/run-124M-COLAB" checkpoint/run-124M-COLAB
# import tensorflow as tf
# tf.reset_default_graph()
# sess = gpt2.start_tf_sess()
# gpt2.load_gpt2(sess, run_name=sess_run_name)

In [ ]:
#@title Generation Parameters
# text_length =  300#@param {type:"number"}
# gen_temp = 0.7 #@param {type:"slider", min:0.7, max:1.0, step:0.01}
# text_prefix = "Gay Conversion Therapists Claim Most Patients Fully Straight By The Time They Commit Suicide" #@param {type:"string"}
# nsamp =  5#@param {type:"number"}
# batch_s =  5#@param {type:"number"}
# gpt2.generate(sess,
#               run_name=sess_run_name,
#               length=text_length,  
#               temperature=gen_temp, 
#               prefix=text_prefix,
#               nsamples=nsamp,
#               batch_size=batch_s 
#               )

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
#gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

#gpt2.generate_to_file(sess,
#                      destination_path=gen_file,
#                      length=500,
#                      temperature=0.7,
#                      nsamples=100,
#                      batch_size=20
#                      )

In [ ]:
# may have to run twice to get file to download
#files.download(gen_file)